In [ ]:
#imports

In [ ]:
#generate "main" gui window in tkinter

In [ ]:
# Instrument Connection
#module to:
# 1. Prompt user to select NPORT port RMA instrument is connected to
# 2. Create a telnet connection to both instruments
# 3. Send wakeup to both instruments
# 4. Alert user that connection successful


In [ ]:
# Collect Samples
#module to:
# 1. Send a wakeup to both instruments
# 2. 